# Juncos Week 5 Assignment
# DS Automation Assignment

Using our prepared churn data from week 2:
- use pycaret to find an ML algorithm that performs best on the data
    - Choose a metric you think is best to use for finding the best model; by default, it is accuracy but it could be AUC, precision, recall, etc. The week 3 FTE has some information on these different metrics.
- save the model to disk
- create a Python script/file/module with a function that takes a pandas dataframe as an input and returns the probability of churn for each row in the dataframe
    - your Python file/function should print out the predictions for new data (new_churn_data.csv)
    - the true values for the new data are [1, 0, 0, 1, 0] if you're interested
- test your Python module and function with the new data, new_churn_data.csv
- write a short summary of the process and results at the end of this notebook
- upload this Jupyter Notebook and Python file to a Github repository, and turn in a link to the repository in the week 5 assignment dropbox

*Optional* challenges:
- return the probability of churn for each new prediction, and the percentile where that prediction is in the distribution of probability predictions from the training dataset (e.g. a high probability of churn like 0.78 might be at the 90th percentile)
- use other autoML packages, such as TPOT, H2O, MLBox, etc, and compare performance and features with pycaret
- create a class in your Python module to hold the functions that you created
- accept user input to specify a file using a tool such as Python's `input()` function, the `click` package for command-line arguments, or a GUI
- Use the unmodified churn data (new_unmodified_churn_data.csv) in your Python script. This will require adding the same preprocessing steps from week 2 since this data is like the original unmodified dataset from week 1.

In [3]:
%pip install pycaret

     |████████████████████████████████| 320 kB 2.2 MB/s            
     |████████████████████████████████| 167 kB 30.8 MB/s            
     |████████████████████████████████| 120 kB 20.2 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 1.7 MB 17.1 MB/s            
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     |████████████████████████████████| 56 kB 7.4 MB/s             
     |████████████████████████████████| 23.4 MB 413 kB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 1.5 MB 18.6 MB/s            
     |████████████████████████████████| 1.3 MB 25.6 MB/s            
     |████████████████████████████████| 1.2 MB 36.2 MB/s            
     |████████████████████████████████| 81 kB 13.3 MB/s            
  Preparing metadata (setup.py) ... done
     |██████

In [59]:
import pandas as pd

In [71]:
# import data where I replaced TotalCharges and tenure null/0 values to median
df = pd.read_csv('prepped_churn_data2.csv', index_col='customerID')
df

,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,Churn,totalcharge_tenure_ratio
customerID,,,,,,,,
7590-VHVEG,1,0,0,0,29.85,29.85,0,29.85
5575-GNVDE,34,1,1,1,56.95,1889.50,0,55.57
3668-QPYBK,2,1,0,1,53.85,108.15,1,54.08
7795-CFOCW,45,0,1,2,42.30,1840.75,0,40.91
9237-HQITU,2,1,0,0,70.70,151.65,1,75.82
...,...,...,...,...,...,...,...,...
6840-RESVB,24,1,1,1,84.80,1990.50,0,82.94
2234-XADUH,72,1,1,3,103.20,7362.90,0,102.26
4801-JZAZL,11,0,0,0,29.60,346.45,0,31.50


## Use pycaret to find an ML algorithm that performs best on the data


In [61]:
# import pycaret libraries
from pycaret.classification import setup, compare_models, predict_model, save_model, load_model

In [62]:
automl = setup(df, target='Churn')

,Description,Value
0,session_id,5900
1,Target,Churn
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(7043, 8)"
5,Missing Values,False
6,Numeric Features,4
7,Categorical Features,3
8,Ordinal Features,False
9,High Cardinality Features,False


In [70]:
# Viewed multiple, but went with Precision. Also, choosing Gradient Boosting Classifier as it is most frequently #1, or close to the top for most metrics
best_model = compare_models(sort='Precision')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ada,Ada Boost Classifier,0.7895,0.8331,0.4840,0.6374,0.5491,0.4154,0.4227,0.0530
gbc,Gradient Boosting Classifier,0.7899,0.8329,0.4878,0.6366,0.5516,0.4178,0.4245,0.0740
ridge,Ridge Classifier,0.7846,0.0000,0.4543,0.6324,0.5283,0.3937,0.4029,0.0100
lr,Logistic Regression,0.7888,0.8315,0.5114,0.6270,0.5627,0.4255,0.4298,0.8400
lightgbm,Light Gradient Boosting Machine,0.7848,0.8232,0.4970,0.6182,0.5506,0.4115,0.4159,0.0320
lda,Linear Discriminant Analysis,0.7832,0.8234,0.5114,0.6103,0.5559,0.4141,0.4173,0.0090
knn,K Neighbors Classifier,0.7629,0.7386,0.4268,0.5733,0.4891,0.3391,0.3455,0.2230
rf,Random Forest Classifier,0.7623,0.7918,0.4627,0.5657,0.5084,0.3539,0.3573,0.1210
et,Extra Trees Classifier,0.7519,0.7644,0.4787,0.5380,0.5063,0.3415,0.3427,0.1250
svm,SVM - Linear Kernel,0.6323,0.0000,0.6407,0.4846,0.4881,0.2625,0.2981,0.0120


In [64]:
best_model

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=50, random_state=5900)

Based on the info above, I will be going with the Gradient Boosting Classifier model as it frequently tops the others on most metrics.

In [65]:
predict_model(best_model, df.iloc[-2:-1])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Ada Boost Classifier,1.0000,0,1.0000,1.0000,1.0000,nan,0.0000


,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,Churn,totalcharge_tenure_ratio,Label,Score
customerID,,,,,,,,,,
8361-LTMKD,4,1,0,1,74.4,306.6,1,76.65,1,0.5005


## Save Model

In [66]:
save_model(best_model, 'GBC')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[],
                                       ml_usecase='classification',
                                       numerical_features=[], target='Churn',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_strate...
                 ('dummy', Dummify(target='Churn')),
                 ('fix_perfect', Remove_100(target='Churn')),
                 ('clean_names', Clean_Colum_Names()),
                 ('feature_select', 'passthrough'), ('fix_multi', 'passthrough'),
                 ('dfs

## Python Module

Created a python module that imports new_churn_data.csv and runs the model against it to determine churn predictions. Note that I needed to change the field charge_per_tenure to totalcharge_tenure_ratio to match the previous data and model.

In [72]:
# Created python module
from IPython.display import Code

Code('predict_churn.py')

import pandas as pd
from pycaret.classification import predict_model, load_model

def load_data(filepath):
    """
    Loads diabetes data into a DataFrame from a string filepath.
    """
    df = pd.read_csv(filepath, index_col='customerID')
    return df


def make_predictions(df):
    """
    Uses the pycaret best model to make predictions on data in the df dataframe.
    """
    model = load_model('GBC')
    predictions = predict_model(model, data=df)
    predictions.rename({'Label': 'Churn_prediction'}, axis=1, inplace=True)
    predictions['Churn_prediction'].replace({1: 'Churn', 0: 'No churn'},
                                            inplace=True)
    return predictions['Churn_prediction']


if __name__ == "__main__":
    df = load_data('new_churn_data.csv')
    predictions = make_predictions(df)
    print('predictions:')
    print(predictions)

#### Now to run the python module.

In [73]:
%run predict_churn.py

Transformation Pipeline and Model Successfully Loaded
predictions:
customerID
9305-CKSKC       Churn
1452-KNGVK    No churn
6723-OKKJM    No churn
7832-POPKP    No churn
6348-TACGU    No churn
Name: Churn_prediction, dtype: object


The results returned by my model are [1, 0, 0, 0, 0]. The actual results are [1, 0, 0, 1, 0]. So there is some error with my model as there is one customer that will Churn, but my model does not catch it.

# Summary

This weeks assignment was to utilize the past data we have and create Models for predictions out of them. I ended up deciding on the Gradient Boosting Classifier model due to it's high precision, accuracy, and the fact that it was the top of the charts across the board. Many times it would even be top in precision, however it currently does not show that.

I also saved this model and created a function that would be able to communicate with this model and compare new_churn_data with it. I had to alter one field label, but I managed to run predictions against the data, of which I found 80% accuracy as the model properly predicted 4/5 of the churn results. There was one missed churn customer. This is accurate with the numbers I have found in previous assignments as I frequently find ~80% accuracy with the models I have created in the past.